#### loading the libraries

In [ ]:
import os
import sys
import pyvista as pv
import trimesh as tm
import numpy as np
import topogenesis as tg
import pickle as pk
import matplotlib.pyplot as plt
sys.path.append(os.path.realpath('..\..')) # no idea how or why this is not working without adding this to the path TODO: learn about path etc.
from notebooks.resources import RES as res
from pygmo import *
import pygmo as pg

#### loading the meshes and vectors

In [ ]:
# load base lattice CSV file
lattice_path = os.path.relpath('../../data/macrovoxels.csv')
macro_lattice = tg.lattice_from_csv(lattice_path)

# load environment
environment_path = os.path.relpath("../../data/movedcontext.obj") 
environment_mesh = tm.load(environment_path)

# load solar vectors
vectors = pk.load(open("../../data/sunvectors.pk", "rb"))

# load vector intensities
intensity_pv = pk.load(open("../../data/ghrval.pk", "rb")) # global horizontal radiation

# load vector intensities
intensity_dl = pk.load(open("../../data/dnival.pk", "rb")) # direct normal illuminance

#### defining target FSI, number of variables

In [ ]:
goal_FSI = 3
num_var = macro_lattice.flatten().shape[0]

#### creating the class for running the optimization

In [ ]:
class pygmo_optimize:

    # Number of dimensions
    def __init__(self, dim, reflattice, crit1_values, crit2_values, sunvectors, environment, targetFSI):
        self.dim = dim
        self.reflattice = reflattice
        self.crit1_values = crit1_values
        self.crit2_values = crit2_values
        self.sunvectors = sunvectors
        self.environment = environment
        self.targetFSI = targetFSI

    # Define objectives    
    def fitness(self, x):
        # PyGmo minimizes. To maximize, we need to invert our objective functions

        # global horizontal radiation on voxel roofs (PV potential) MAXIMIZE
        f1 = - res.crit_1_PV(x, self.reflattice, self.sunvectors, self.crit1_values, self.environment)[0]

        # direct normal illuminance on voxel facade (daylight potential) MAXIMIZE
        f2 = - res.crit_2_DL(x, self.reflattice, self.sunvectors, self.crit2_values, self.environment)[0]

        # relative compactness MINIMIZE
        f3 =   res.crit_3_RC(x, self.reflattice)

        # Floor Space Index deviation MAXIMIZE
        f4 = - res.crit_4_FSI(x, self.reflattice, self.targetFSI)

        return [f1, f2, f3, f4]
    
    # Return number of objectives
    def get_nobj(self):
        return 4

    # Return bounds of decision variables
    def get_bounds(self):
        return (np.full((self.dim,),0.),np.full((self.dim,),1.))

    # Return function name
    def get_name(self):
        return "Test function MAX"

In [ ]:
# create User Defined Problem
prob = pg.problem(pygmo_optimize(
    dim= num_var, 
    reflattice= macro_lattice, 
    crit1_values= intensity_pv, 
    crit2_values= intensity_dl, 
    sunvectors= vectors, 
    environment= environment_mesh, 
    targetFSI= goal_FSI
    ))

#### the setup

In [ ]:
pop_sizes=[64, 128]
udp = prob

# hv_nspso=[0, 0]
# hv_ihs=[0, 0]
# hv_nsga2=[0, 0]

# p_dist_nspso=[0, 0]
# p_dist_ihs=[0, 0]
# p_dist_nsga2=[0, 0]

In [ ]:
# We run the algos 2 times each, for 2 different pop-sizes

for j in pop_sizes:

    pop_1 = pg.population(prob = udp, size = j, seed = 0)
    pop_2 = pg.population(prob = udp, size = j, seed = 0)
    pop_3 = pg.population(prob = udp, size = j, seed = 0)


    # initial seed:

    if j==pop_sizes[0]:

        first_pop_low_1=pop_1.get_f()
        first_pop_low_2=pop_1.get_f()
        first_pop_low_3=pop_1.get_f()

    if j==pop_sizes[1]:

        first_pop_high_1=pop_1.get_f()
        first_pop_high_2=pop_1.get_f()
        first_pop_high_3=pop_1.get_f()

    # the algorithms used
    algo = pg.algorithm(pg.nspso(gen = 250))
    algo_2 = pg.algorithm(pg.ihs(gen = 250))
    algo_3 = pg.algorithm(pg.nsga2(gen = 250))

    # setting the algorithm seed
    algo.set_seed(0)
    algo_2.set_seed(0)
    algo_3.set_seed(0)

    # calling the functions
    pop_1 = algo.evolve(pop_1)
    pop_2= algo_2.evolve(pop_2)
    pop_3 = algo_3.evolve(pop_3)


    #This returns a series of arrays: in each of them it is contained (in this order), the -non-dominated front, -domination list,

    #-domination count, -non-domination rank

    fnds= pg.fast_non_dominated_sorting(pop_1.get_f())
    fnds_2= pg.fast_non_dominated_sorting(pop_2.get_f())
    fnds_3= pg.fast_non_dominated_sorting(pop_3.get_f())


    #This returns the first (i.e., best) non-dominated front:

    first_ndf_nspso=fnds[0][0]
    first_ndf_ihs=fnds_2[0][0]
    first_ndf_nsga2=fnds_3[0][0]


    # Store all the pop-sizes non-dominated fronts for all the runs:

    if j==pop_sizes[0]:

        # NSPSO

        # hv_nspso[0]= pg.hypervolume(pop_1).compute(ref_point)
        # p_dist_nspso[0]=udp.p_distance(pop_1)

        first_col_nspso_poplow_1=pop_1.get_f()[first_ndf_nspso,0]

        second_col_nspso_poplow_1=pop_1.get_f()[first_ndf_nspso,1]

        third_col_nspso_poplow_1=pop_1.get_f()[first_ndf_nspso,2]

        fourth_col_nspso_poplow_1=pop_1.get_f()[first_ndf_nspso,3]

        # IHS

        # hv_ihs[0]= pg.hypervolume(pop_2).compute(ref_point)
        # p_dist_ihs[0]=udp.p_distance(pop_2)

        first_col_ihs_poplow_1=pop_2.get_f()[first_ndf_ihs,0]

        second_col_ihs_poplow_1=pop_2.get_f()[first_ndf_ihs,1]

        third_col_ihs_poplow_1=pop_2.get_f()[first_ndf_ihs,2]

        fourth_col_ihs_poplow_1=pop_2.get_f()[first_ndf_ihs,3]

        # NSGA2

        # hv_nsga2[0]= pg.hypervolume(pop_3).compute(ref_point)
        # p_dist_nsga2[0]=udp.p_distance(pop_3)

        first_col_nsga2_poplow_1=pop_3.get_f()[first_ndf_nsga2,0]

        second_col_nsga2_poplow_1=pop_3.get_f()[first_ndf_nsga2,1]

        third_col_nsga2_poplow_1=pop_3.get_f()[first_ndf_nsga2,2]

        fourth_col_nsga2_poplow_1=pop_3.get_f()[first_ndf_nsga2,3]


    if j==pop_sizes[1]:

        # NSPSO

        # hv_nspso[1]= pg.hypervolume(pop_1).compute(ref_point)
        # p_dist_nspso[1]=udp.p_distance(pop_1)

        first_col_nspso_pophigh_1=pop_1.get_f()[first_ndf_nspso,0]

        second_col_nspso_pophigh_1=pop_1.get_f()[first_ndf_nspso,1]

        third_col_nspso_pophigh_1=pop_1.get_f()[first_ndf_nspso,2]

        fourth_col_nspso_pophigh_1=pop_1.get_f()[first_ndf_nspso,3]

        # IHS

        # hv_ihs[1]= pg.hypervolume(pop_2).compute(ref_point)
        # p_dist_ihs[1]=udp.p_distance(pop_2)

        first_col_ihs_pophigh_1=pop_2.get_f()[first_ndf_ihs,0]

        second_col_ihs_pophigh_1=pop_2.get_f()[first_ndf_ihs,1]

        third_col_ihs_pophigh_1=pop_2.get_f()[first_ndf_ihs,2]

        fourth_col_ihs_pophigh_1=pop_2.get_f()[first_ndf_ihs,3]

        # NSGA2

        # hv_nsga2[1]= pg.hypervolume(pop_3).compute(ref_point)
        # p_dist_nsga2[1]=udp.p_distance(pop_3)

        first_col_nsga2_pophigh_1=pop_3.get_f()[first_ndf_nsga2,0]

        second_col_nsga2_pophigh_1=pop_3.get_f()[first_ndf_nsga2,1]

        third_col_nsga2_pophigh_1=pop_3.get_f()[first_ndf_nsga2,2]

        fourth_col_nsga2_pophigh_1=pop_3.get_f()[first_ndf_nsga2,3]


In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, sharex='row', sharey='row', figsize=(15,15)) 


# axes[0,0].plot(first_pop_low_1[:,0], first_pop_low_1[:,1], '.', label= 'initial population') 

axes[0,0].plot(first_col_nspso_poplow_1, second_col_nspso_poplow_1,'kx', label = 'nspso') 

axes[0,0].plot(first_col_ihs_poplow_1, second_col_ihs_poplow_1,'ro', label = 'ihs') 

axes[0,0].plot(first_col_nsga2_poplow_1, second_col_nsga2_poplow_1, 'b^', label = 'nsga2') 

axes[0,0].legend(loc='upper right') 

axes[0,0].set_title('final Pareto front: 250 gens f1 vs f2, pop=64') 

# axes[0,1].plot(first_pop_high_1[:,0], first_pop_high_1[:,1], '.', label= '50 pop') 
axes[0,1].plot(np.concatenate((first_col_nspso_poplow_1,first_col_ihs_poplow_1,first_col_nsga2_poplow_1)), 
               np.concatenate((second_col_nspso_poplow_1,second_col_ihs_poplow_1,second_col_nsga2_poplow_1)), '.', label= '64 pop solutions') 

axes[0,1].plot(first_col_nspso_pophigh_1, second_col_nspso_pophigh_1,'kx', label = 'nspso') 

axes[0,1].plot(first_col_ihs_pophigh_1, second_col_ihs_pophigh_1,'ro', label = 'ihs') 

axes[0,1].plot(first_col_nsga2_pophigh_1, second_col_nsga2_pophigh_1, 'b^', label = 'nsga2') 

axes[0,1].legend(loc='upper right') 

axes[0,1].set_title('final Pareto front: 250 gens f1 vs f2, pop=128') 



# axes[1,0].plot(first_pop_low_2[:,0], first_pop_low_2[:,1], '.', label= 'initial population') 

axes[1,0].plot(first_col_nspso_poplow_1, third_col_nspso_poplow_1,'kx', label = 'nspso') 

axes[1,0].plot(first_col_ihs_poplow_1, third_col_ihs_poplow_1,'ro', label = 'ihs') 

axes[1,0].plot(first_col_nsga2_poplow_1, third_col_nsga2_poplow_1, 'b^', label = 'nsga2') 

axes[1,0].legend(loc='upper right') 

axes[1,0].set_title('final Pareto front: 250 gens f1 vs f3, pop=64') 

# axes[1,1].plot(first_pop_high_2[:,0], first_pop_high_2[:,1], '.', label= 'initial population') 
axes[1,1].plot(np.concatenate((first_col_nspso_poplow_1,first_col_ihs_poplow_1,first_col_nsga2_poplow_1)), 
               np.concatenate((third_col_nspso_poplow_1,third_col_ihs_poplow_1,third_col_nsga2_poplow_1)), '.', label= '64 pop solutions') 

axes[1,1].plot(first_col_nspso_pophigh_1, third_col_nspso_pophigh_1,'kx', label = 'nspso') 

axes[1,1].plot(first_col_ihs_pophigh_1, third_col_ihs_pophigh_1,'ro', label = 'ihs') 

axes[1,1].plot(first_col_nsga2_pophigh_1, third_col_nsga2_pophigh_1, 'b^', label = 'nsga2') 

axes[1,1].legend(loc='upper right') 

axes[1,1].set_title('final Pareto front: 250 gens f1 vs f3, pop=128') 



# axes[2,0].plot(first_pop_low_3[:,0], first_pop_low_3[:,1], '.', label= 'initial population') 

axes[2,0].plot(second_col_nspso_poplow_1, third_col_nspso_poplow_1,'kx', label = 'nspso') 

axes[2,0].plot(second_col_ihs_poplow_1, third_col_ihs_poplow_1,'ro', label = 'ihs') 

axes[2,0].plot(second_col_nsga2_poplow_1, third_col_nsga2_poplow_1, 'b^', label = 'nsga2') 

axes[2,0].legend(loc='upper right') 

axes[2,0].set_title('final Pareto front: 250 gens f2 vs f3, pop=64') 

# axes[2,1].plot(first_pop_high_3[:,0], first_pop_high_3[:,1], '.', label= 'initial population') 
axes[2,1].plot(np.concatenate((second_col_nspso_poplow_1,second_col_ihs_poplow_1,second_col_nsga2_poplow_1)), 
               np.concatenate((third_col_nspso_poplow_1,third_col_ihs_poplow_1,third_col_nsga2_poplow_1)), '.', label= '64 pop solutions') 

axes[2,1].plot(second_col_nspso_pophigh_1, third_col_nspso_pophigh_1,'kx', label = 'nspso') 

axes[2,1].plot(second_col_ihs_pophigh_1, third_col_ihs_pophigh_1,'ro', label = 'ihs') 

axes[2,1].plot(second_col_nsga2_pophigh_1, third_col_nsga2_pophigh_1, 'b^', label = 'nsga2') 

axes[2,1].legend(loc='upper right') 

axes[2,1].set_title('final Pareto front: 250 gens f2 vs f3, pop=128') 

plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.2, 
                    hspace=0.4)

for ax in axes[0]: 

   ax.set(xlabel='f_1 PV potential (Wh)', ylabel='f_2 DL potential (hectolux)') 

   ax.grid() 

for ax in axes[1]: 

   ax.set(xlabel='f_1 PV potential (Wh)', ylabel='f_3 HR potential (1 is perfect cube)') 

   ax.grid() 

for ax in axes[2]: 

   ax.set(xlabel='f_2 DL potential (hectolux)', ylabel='f_3 HR potential (1 is perfect cube)') 

   ax.grid() 